In [167]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [168]:
df  = pd.read_csv('../data/hawkes_analysis_data.csv')

In [169]:
df.head()

,trade_number,trade_price,volume,buy_order_number,buy_algo,buy_client,sell_order_number,sell_algo,sell_client,timestamp,buy_entry_ts,sell_entry_ts,aggressor_side,date
0,2019081325008066,785.0,1,1100000000068964,1,2,1100000000062580,1,3,2019-08-13 09:15:00.006118774,2019-08-13 09:15:00.005767822,NaN,-1,2019-08-13
1,2019081325008128,785.0,2,1100000000069441,1,3,1100000000062580,1,3,2019-08-13 09:15:00.024169922,2019-08-13 09:15:00.024139404,NaN,-1,2019-08-13
2,2019081325008129,785.0,47,1100000000069442,1,3,1100000000062580,1,3,2019-08-13 09:15:00.024230957,2019-08-13 09:15:00.024215698,NaN,-1,2019-08-13
3,2019081325008130,785.0,53,1100000000069442,1,3,1100000000068983,0,3,2019-08-13 09:15:00.024246216,2019-08-13 09:15:00.024215698,2019-08-13 09:15:00.005935669,1,2019-08-13
4,2019081325008466,785.0,10,1100000000055032,1,3,1100000000070554,0,1,2019-08-13 09:15:00.118347168,NaN,2019-08-13 09:15:00.118331909,1,2019-08-13


In [175]:
df_day['timestamp'] = pd.to_datetime(df_day['timestamp'])

In [170]:
dates = df['date'].unique()

In [171]:
target_date = dates[0]

In [172]:
df_day = df[df['date'] == target_date].sort_values('timestamp')

In [176]:
df_day['timestamp'].dtype

dtype('<M8[ns]')

In [179]:
market_open = pd.to_datetime((target_date) + ' 09:15:00')
df_day['t'] = (df_day['timestamp'] - market_open).dt.total_seconds()

In [182]:
buy_times = df_day[df_day['aggressor_side'] == +1]['t'].to_numpy()
sell_times = df_day[df_day['aggressor_side'] == -1]['t'].to_numpy()

events = [buy_times, sell_times]

In [183]:
decay = 1/60 

In [185]:
from tick.hawkes import HawkesExpKern

hawkes = HawkesExpKern(decays=decay, penalty='l2', C=1e-3)

ModuleNotFoundError: No module named 'tick'